### 서울시 데이터 통합

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### 1. CCTV 데이터

In [3]:
# 1) 데이터 파일 읽기
filename = 'data/서울시 자치구 년도별 CCTV 설치 현황_221231기준.csv'
columns = '구분,총계,2013년 이전 설치된 CCTV,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년'.split(',')
cctv = pd.read_csv(filename, encoding='euc=kr', thousands=',')
cctv = cctv[columns]
cctv.head()

,구분,총계,2013년 이전 설치된 CCTV,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
0,계,91341,2804,3097,3981,6576,8850,11566,11024,13151,11912,9144,9236
1,종로구,1980,36,540,107,161,131,158,152,69,250,85,291
2,중 구,2584,130,87,77,236,240,372,386,155,361,403,137
3,용산구,2847,44,50,68,83,295,491,115,322,623,422,334
4,성동구,4047,58,99,110,366,279,945,459,647,485,367,232


In [4]:
# 2) 결측치 확인
cctv.isna().sum().sum()

0

In [5]:
# 3) 최근 증가율
cctv.loc[:, '2020년':'2022년'].sum(axis=1).head()

0    30292
1      626
2      901
3     1379
4     1084
dtype: int64

In [6]:
cctv['최근증가율'] = cctv.iloc[:, 10:].sum(axis=1) / cctv.iloc[:, 2:10].sum(axis=1)
cctv['최근증가율'] = (cctv['최근증가율'] * 100).round(2)
cctv.head()

,구분,총계,2013년 이전 설치된 CCTV,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년,최근증가율
0,계,91341,2804,3097,3981,6576,8850,11566,11024,13151,11912,9144,9236,49.62
1,종로구,1980,36,540,107,161,131,158,152,69,250,85,291,46.23
2,중 구,2584,130,87,77,236,240,372,386,155,361,403,137,53.54
3,용산구,2847,44,50,68,83,295,491,115,322,623,422,334,93.94
4,성동구,4047,58,99,110,366,279,945,459,647,485,367,232,36.58


In [7]:
# 4) Selection and rename
cctv = cctv[['구분', '총계', '최근증가율']]
cctv.head()

,구분,총계,최근증가율
0,계,91341,49.62
1,종로구,1980,46.23
2,중 구,2584,53.54
3,용산구,2847,93.94
4,성동구,4047,36.58


In [8]:
cctv.rename(columns={'구분':'구별', '총계':'CCTV댓수'}, inplace=True)
cctv.head()

,구별,CCTV댓수,최근증가율
0,계,91341,49.62
1,종로구,1980,46.23
2,중 구,2584,53.54
3,용산구,2847,93.94
4,성동구,4047,36.58


In [9]:
# 5) Filtering - 계 데이터 삭제
cctv.drop([0], inplace=True)
cctv.head()

,구별,CCTV댓수,최근증가율
1,종로구,1980,46.23
2,중 구,2584,53.54
3,용산구,2847,93.94
4,성동구,4047,36.58
5,광진구,3480,46.53


In [10]:
# 6) 구 이름에서 공백 지우기
cctv.구별 = cctv.구별.str.replace(' ', '')
cctv.head(3)

,구별,CCTV댓수,최근증가율
1,종로구,1980,46.23
2,중구,2584,53.54
3,용산구,2847,93.94


In [11]:
# 7) cctv.csv 파일에 저장
cctv.to_csv('data/cctv.csv', index=False)
df = pd.read_csv('data/cctv.csv')
df.head()

,구별,CCTV댓수,최근증가율
0,종로구,1980,46.23
1,중구,2584,53.54
2,용산구,2847,93.94
3,성동구,4047,36.58
4,광진구,3480,46.53


### 2. 인구 데이터

In [12]:
# 1) 파일 읽기
pop = pd.read_csv('data/서울시인구.txt', sep='\t')
pop.head()

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
0,기간,자치구,세대,합계,합계,합계,한국인,한국인,한국인,등록외국인,등록외국인,등록외국인,세대당인구,65세이상고령자
1,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,세대당인구,65세이상고령자
2,2021,합계,"4,426,007","9,736,027","4,721,977","5,014,050","9,509,458","4,618,040","4,891,418","226,569","103,937","122,632",2.15,"1,605,416"
3,2021,종로구,"73,494","153,789","74,186","79,603","144,683","70,183","74,500","9,106","4,003","5,103",1.97,"27,818"
4,2021,중구,"63,519","131,787","64,083","67,704","122,499","59,630","62,869","9,288","4,453","4,835",1.93,"24,392"


In [13]:
pop = pd.read_csv('data/서울시인구.txt', sep='\t', skiprows=2)
pop.head()

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2021,합계,"4,426,007","9,736,027","4,721,977","5,014,050","9,509,458","4,618,040","4,891,418","226,569","103,937","122,632",2.15,"1,605,416"
1,2021,종로구,"73,494","153,789","74,186","79,603","144,683","70,183","74,500","9,106","4,003","5,103",1.97,"27,818"
2,2021,중구,"63,519","131,787","64,083","67,704","122,499","59,630","62,869","9,288","4,453","4,835",1.93,"24,392"
3,2021,용산구,"111,036","237,285","115,085","122,200","222,953","107,210","115,743","14,332","7,875","6,457",2.01,"39,070"
4,2021,성동구,"134,233","292,672","142,259","150,413","285,990","139,380","146,610","6,682","2,879","3,803",2.13,"46,380"


In [14]:
# 2) 결측치 확인
pop.isna().sum().sum()

0

In [15]:
# 3) Selection and rename
pop = pop[['자치구','계','계.1','계.2','65세이상고령자']]
pop.head(3)

,자치구,계,계.1,계.2,65세이상고령자
0,합계,"9,736,027","9,509,458","226,569","1,605,416"
1,종로구,"153,789","144,683","9,106","27,818"
2,중구,"131,787","122,499","9,288","24,392"


In [16]:
pop.columns = ['구별','인구수','내국인','외국인','고령자']
pop.head(3)

,구별,인구수,내국인,외국인,고령자
0,합계,"9,736,027","9,509,458","226,569","1,605,416"
1,종로구,"153,789","144,683","9,106","27,818"
2,중구,"131,787","122,499","9,288","24,392"


In [17]:
# 4) 합계 데이터 삭제
pop.drop([0], inplace=True)
pop.head(3)

,구별,인구수,내국인,외국인,고령자
1,종로구,"153,789","144,683","9,106","27,818"
2,중구,"131,787","122,499","9,288","24,392"
3,용산구,"237,285","222,953","14,332","39,070"


In [18]:
# 5) 천단위 구분기호 없애고 정수로 변환하기
for column in pop.columns[1:]:
    pop[column] = pop[column].str.replace(',','').astype(int)
pop.head(3)

,구별,인구수,내국인,외국인,고령자
1,종로구,153789,144683,9106,27818
2,중구,131787,122499,9288,24392
3,용산구,237285,222953,14332,39070


In [19]:
# 6) 외국인 비율, 고령자 비율 컬럼 만들기
pop['외국인비율'] = (pop.외국인 / pop.인구수 * 100).round(2)
pop['고령자비율'] = (pop.고령자 / pop.인구수 * 100).round(2)
pop.head(3)

,구별,인구수,내국인,외국인,고령자,외국인비율,고령자비율
1,종로구,153789,144683,9106,27818,5.92,18.09
2,중구,131787,122499,9288,24392,7.05,18.51
3,용산구,237285,222953,14332,39070,6.04,16.47


In [20]:
# 7) pop.csv에 저장하기
pop.to_csv('data/pop.csv', index=False)
df = pd.read_csv('data/pop.csv')
df.head(3)

,구별,인구수,내국인,외국인,고령자,외국인비율,고령자비율
0,종로구,153789,144683,9106,27818,5.92,18.09
1,중구,131787,122499,9288,24392,7.05,18.51
2,용산구,237285,222953,14332,39070,6.04,16.47


### 3. 두 데이터 병합

In [21]:
df = pd.merge(cctv, pop)
df

,구별,CCTV댓수,최근증가율,인구수,내국인,외국인,고령자,외국인비율,고령자비율
0,종로구,1980,46.23,153789,144683,9106,27818,5.92,18.09
1,중구,2584,53.54,131787,122499,9288,24392,7.05,18.51
2,용산구,2847,93.94,237285,222953,14332,39070,6.04,16.47
3,성동구,4047,36.58,292672,285990,6682,46380,2.28,15.85
4,광진구,3480,46.53,352627,339996,12631,51723,3.58,14.67
5,동대문구,2759,34.19,352006,337400,14606,62211,4.15,17.67
6,중랑구,4193,73.55,391885,387350,4535,71682,1.16,18.29
7,성북구,4842,25.96,440142,430528,9614,74709,2.18,16.97
8,강북구,3321,100.42,302563,299182,3381,64333,1.12,21.26
9,도봉구,2247,155.92,319373,317366,2007,64160,0.63,20.09


In [22]:
df.set_index('구별', inplace=True)
df

,CCTV댓수,최근증가율,인구수,내국인,외국인,고령자,외국인비율,고령자비율
구별,,,,,,,,
종로구,1980,46.23,153789,144683,9106,27818,5.92,18.09
중구,2584,53.54,131787,122499,9288,24392,7.05,18.51
용산구,2847,93.94,237285,222953,14332,39070,6.04,16.47
성동구,4047,36.58,292672,285990,6682,46380,2.28,15.85
광진구,3480,46.53,352627,339996,12631,51723,3.58,14.67
동대문구,2759,34.19,352006,337400,14606,62211,4.15,17.67
중랑구,4193,73.55,391885,387350,4535,71682,1.16,18.29
성북구,4842,25.96,440142,430528,9614,74709,2.18,16.97
강북구,3321,100.42,302563,299182,3381,64333,1.12,21.26


In [23]:
df.to_csv('data/cctv_pop.csv')
vf = pd.read_csv('data/cctv_pop.csv')
vf.head()

,구별,CCTV댓수,최근증가율,인구수,내국인,외국인,고령자,외국인비율,고령자비율
0,종로구,1980,46.23,153789,144683,9106,27818,5.92,18.09
1,중구,2584,53.54,131787,122499,9288,24392,7.05,18.51
2,용산구,2847,93.94,237285,222953,14332,39070,6.04,16.47
3,성동구,4047,36.58,292672,285990,6682,46380,2.28,15.85
4,광진구,3480,46.53,352627,339996,12631,51723,3.58,14.67


---

In [24]:
filename = 'data/주민등록인구집계현황.csv'
columns = '행정구역구분명,행정구역명,60~69세,70~79세,80~89세,90~99세,100세 이상,20~29세 (여),30~39세 (여)'.split(',')
ggpop = pd.read_csv(filename, encoding='euc=kr',)
ggpop = ggpop[columns]
ggpop.head()

,행정구역구분명,행정구역명,60~69세,70~79세,80~89세,90~99세,100세 이상,20~29세 (여),30~39세 (여)
0,도,경기도,1750962,808663,403721,58065,1725,821150,904073
1,시군,경기도 가평군,13315,7082,3999,564,19,2503,2433
2,읍면동,경기도 가평군 가평읍,3614,1963,1206,163,7,937,848
3,읍면동,경기도 가평군 북면,1072,578,353,54,1,87,99
4,읍면동,경기도 가평군 상면,1537,788,410,57,0,151,166


In [25]:
ggpop.isna().sum().sum()

0

In [26]:
ggpop['가임기 여성인구'] = ggpop.iloc[:, 7:9].sum(axis=1)
ggpop['60세 이상 노령인구'] = ggpop.iloc[:, 2:7].sum(axis=1)
ggpop['인구소멸위기지역'] = (ggpop['가임기 여성인구']/ggpop['60세 이상 노령인구']).round(2)
ggpop.head()

,행정구역구분명,행정구역명,60~69세,70~79세,80~89세,90~99세,100세 이상,20~29세 (여),30~39세 (여),가임기 여성인구,60세 이상 노령인구,인구소멸위기지역
0,도,경기도,1750962,808663,403721,58065,1725,821150,904073,1725223,3023136,0.57
1,시군,경기도 가평군,13315,7082,3999,564,19,2503,2433,4936,24979,0.20
2,읍면동,경기도 가평군 가평읍,3614,1963,1206,163,7,937,848,1785,6953,0.26
3,읍면동,경기도 가평군 북면,1072,578,353,54,1,87,99,186,2058,0.09
4,읍면동,경기도 가평군 상면,1537,788,410,57,0,151,166,317,2792,0.11


In [27]:
ggpop = ggpop[['행정구역구분명','행정구역명','가임기 여성인구','60세 이상 노령인구','인구소멸위기지역']]
ggpop.head()

,행정구역구분명,행정구역명,가임기 여성인구,60세 이상 노령인구,인구소멸위기지역
0,도,경기도,1725223,3023136,0.57
1,시군,경기도 가평군,4936,24979,0.20
2,읍면동,경기도 가평군 가평읍,1785,6953,0.26
3,읍면동,경기도 가평군 북면,186,2058,0.09
4,읍면동,경기도 가평군 상면,317,2792,0.11


In [32]:
ggpop = ggpop[ggpop.행정구역구분명.str.contains('시군') | ggpop.행정구역구분명.str.contains('구')]
ggpop.head()

,행정구역구분명,행정구역명,가임기 여성인구,60세 이상 노령인구,인구소멸위기지역
1,시군,경기도 가평군,4936,24979,0.20
8,시군,경기도 고양시,140519,246509,0.57
9,구,경기도 고양시 덕양구,64639,116439,0.56
31,구,경기도 고양시 일산동구,39716,65674,0.60
44,구,경기도 고양시 일산서구,36164,64396,0.56


In [36]:
city = ['수원시','고양시','용인시','성남시','안산시','안양시']
ggpop.행정구역명 = ggpop.행정구역명.str.strip()
drop_index = []
for index in ggpop[ggpop.행정구역구분명 == '시군'].index:
    if ggpop.행정구역명[index].split()[-1] in city:
        drop_index.append(index)
drop_index

[8, 176, 231, 300, 344, 435]

In [37]:
ggpop.drop(drop_index, inplace=True)
ggpop.head()

,행정구역구분명,행정구역명,가임기 여성인구,60세 이상 노령인구,인구소멸위기지역
1,시군,경기도 가평군,4936,24979,0.20
9,구,경기도 고양시 덕양구,64639,116439,0.56
31,구,경기도 고양시 일산동구,39716,65674,0.60
44,구,경기도 고양시 일산서구,36164,64396,0.56
56,시군,경기도 과천시,10577,16955,0.62


In [38]:
ggpop.sort_values(by='인구소멸위기지역').head(10)

,행정구역구분명,행정구역명,가임기 여성인구,60세 이상 노령인구,인구소멸위기지역
1,시군,경기도 가평군,4936,24979,0.20
417,시군,경기도 연천군,3392,16705,0.20
391,시군,경기도 양평군,9553,47974,0.20
404,시군,경기도 여주시,10399,38716,0.27
558,시군,경기도 포천시,13604,49130,0.28
156,시군,경기도 동두천시,9316,28663,0.33
328,시군,경기도 안성시,20043,52478,0.38
464,구,경기도 용인시 처인구,30288,62462,0.48
363,구,경기도 안양시 만안구,30705,63628,0.48
218,구,경기도 성남시 중원구,27357,55095,0.50


In [39]:
ggpop.sort_values(by='인구소멸위기지역', ascending=False).head(10)

,행정구역구분명,행정구역명,가임기 여성인구,60세 이상 노령인구,인구소멸위기지역
589,시군,경기도 화성시,121045,139409,0.87
428,시군,경기도 오산시,29847,39087,0.76
280,시군,경기도 시흥시,65483,90686,0.72
574,시군,경기도 하남시,44804,67167,0.67
452,구,경기도 용인시 수지구,48401,72918,0.66
177,구,경기도 성남시 분당구,65679,98851,0.66
532,시군,경기도 평택시,73311,111976,0.65
301,구,경기도 안산시 단원구,39514,61472,0.64
345,구,경기도 안양시 동안구,42279,66941,0.63
232,구,경기도 수원시 권선구,42151,67283,0.63
